### 수수료 엑셀을 읽어 지급율 row를 만든다

In [2]:
import pandas as pd

### mp서식을 읽는 방법

In [86]:
지점='표준'
대상='사원'
일괄적용율=1
보종='생보'

path='E:/글로벌백업(180329)/03.수수료&시상/수수료규정/수수료개정_2021/'
in_file='2021년 생명보험 수수료_20210201.xlsx'
# in_file='2021년 손해보험 수수료(20210210).xlsx'
sheet='회사'

c='회사 보종 환산율 상품명 1회 1차년율 1차년회차 2차년율 2차년회차 3차년율 3차년회차 tot'
columns=c.split(' ')
use_cols='a:d,e,g:h,l:m,q:r,v'
df=pd.read_excel(path+in_file,sheet_name=sheet,header=4,usecols=use_cols,dtype={'a': 'str', 'b': 'str'})
df.columns=columns
df['회사'].fillna(method='ffill',inplace=True)
df['보종'].fillna(method='ffill',inplace=True)
df['tot'].fillna(method='ffill',inplace=True)
df.loc[:,'회사']= df['회사'].str.replace('\n','')
df.fillna(0,inplace=True )

In [87]:
''' 서식에 대한 전처리'''
key_col=['회사','보종','tot','1회']
df_t=df[key_col]
df_t.loc[:,'회차']='1회'

key_col=['회사','보종','tot','회차','율']
df_t.rename(columns={'1회':'율'},inplace=True)
df_rate=df_t[key_col]

key_col=['회사','보종','tot','1차년회차','1차년율']
df_t=df[key_col].rename(columns={'1차년회차':'회차','1차년율':'율'})
df_rate=pd.concat([df_rate,df_t])

key_col=['회사','보종','tot','2차년회차','2차년율']
df_t=df[key_col].rename(columns={'2차년회차':'회차','2차년율':'율'})
df_rate=pd.concat([df_rate,df_t])

key_col=['회사','보종','tot','3차년회차','3차년율']
df_t=df[key_col].rename(columns={'3차년회차':'회차','3차년율':'율'})
df_rate=pd.concat([df_rate,df_t])

df_rate=df_rate[df_rate['율']!=0]
df_rate['회사율']=df_rate['율']
df_rate['율']= round(df_rate['율']*일괄적용율,4)
df_rate.loc[:,'회차']= [str(x).replace('회','') for x in df_rate['회차']] 

In [88]:
'''각 회차별 생성'''
df_all=pd.DataFrame()
for i,row in df_rate.iterrows():
    for cnt in str(row['회차']).split(','):
        cnt1=cnt;  cnt2=cnt
        if '~' in cnt:
            cnt1,cnt2=cnt.split('~') 
        row['회차']=cnt1
        row['회차T']=cnt2
        df_all=df_all.append(row)

In [89]:
'''출력을 위한 파생항목 생성'''
df_rate_tbl=df_all
df_rate_tbl=df_rate_tbl.astype({'회차':'int64','회차T':'int64'})
df_rate_tbl['구분']=대상
df_rate_tbl['지점']=지점
df_rate_tbl = df_rate_tbl[['구분','회사','보종','율','회차','회차T','지점','회사율','tot']]

df_rate_tbl.sort_values(by=['회사','보종','회차','회차T'],ascending=[False,True,True,True],inplace=True)
df_rate_tbl['temp']=df_rate_tbl['회사율']*(df_rate_tbl['회차T']-df_rate_tbl['회차']+1)
df_rate_tbl['tot_compare']=df_rate_tbl.groupby(['회사','보종']).temp.transform('sum')
df_rate_tbl.drop('temp',axis=1, inplace=True)

'''tot와 tot_compare가 다르면 엑셀 수식오류 검증해봐야함'''

outFile=f'E:/temp/{지점}_{보종} 지급율.xlsx' 
with pd.ExcelWriter(outFile) as writer:  
    df_rate_tbl.to_excel(writer, sheet_name='지급율',index=False)

### 아래는 mp서식읽은 결과로 glad방식(회차별로 column)을 만드는 작업

In [ ]:
df_all=pd.DataFrame()
for i,row in df_rate.iterrows():
    for cnt in str(row['회차']).split(','):
        cnt1=cnt;  cnt2=cnt
        if '~' in cnt:
            cnt1,cnt2=cnt.split('~') 
        for x in range(int(cnt1),int(cnt2)+1):
            row['회차'] = x
            df_all=df_all.append(row)
        

In [ ]:
df_rate_tbl=df_all
df_rate_tbl=df_rate_tbl.astype({'회차':'int64'})

df_rate_tbl['구분']=대상
df_rate_tbl['지점']=지점
df_rate_tbl = df_rate_tbl[['구분','회사','보종','율','회차']]

df_rate_tbl.sort_values(by=['회사','보종'],ascending=[False,True],inplace=True)
df_rate_tbl = pd.pivot_table(df_rate_tbl,index=['구분','회사','보종'],columns='회차',values=['율'])
df_rate_tbl = df_rate_tbl.fillna(0)            
df_rate_tbl.reset_index(inplace=True)
df_rate_tbl.columns=df_rate_tbl.columns.get_level_values(1) #multi index -> single index

outFile=f'E:/temp/{지점}_{보종} 회차별지급율.xlsx' 
with pd.ExcelWriter(outFile) as writer:  
    df_rate_tbl.to_excel(writer, sheet_name='지급율',index=False)

### 수수료 파일 비교

In [ ]:
''' 두개의 수수료 파일 총 지급율을 비교하여 차이나는 건 추출'''
dt2=df_rate2.groupby(['회사','보종'])['회사율'].sum().reset_index()
dt1=df_rate1.groupby(['회사','보종'])['회사율'].sum().reset_index()
dt_merge= pd.merge(dt2,dt1,on=['회사','보종'],how='left')
dt_merge[dt_merge['회사율_x'] != dt_merge['회사율_y']]

### glad표준파일

In [60]:
## glad표준 엑셀 파일
지점='조은'
대상='사원'
일괄적용율=1.0
보종='손보'

path='E:/글로벌백업(180329)/03.수수료&시상/수수료규정/수수료개정_2021/지점별/'
in_file='엠피조은_생손보수수료_202101.xlsx'
sheet='엠피조은- 손보 - 이설화,김성희 '
c='회사 보종 상품명 납기 환산율 환산율2 환산율3 1회 수입P1 2회 3회 4회 5회 6회 7회 8회 9회 10회 11회 12회 지급계1y 수입p1y 13회 14회 15회 16회 17회 18회 19회 20회 21회 22회 23회 24회 지급계2y 수입P2y 25회 26회 27회 28회 29회 30회 31회 32회 33회 34회 35회 36회 지급계3y 수입P3y 지급계 수입P 비고'
columns=c.split(' ')
df=pd.read_excel(path+in_file,sheet_name=sheet,header=4,names=columns)
df['회사'].fillna(method='ffill',inplace=True)
df.fillna(0,inplace=True )

In [8]:
# df=df.iloc[35:47]
# 컬럼중 회차별 지급율이 있는 컬럼만 추출-- 키워드는 '회'
value_col=[x for x in columns if x.endswith('회') ]
df_rate=df.melt(id_vars=['회사','보종'],value_vars=value_col,var_name='회차',value_name='율')
df_rate=df_rate[df_rate['율']!=0]     #지급율 없으면 제거
df_rate['회사율']=df_rate['율']
df_rate['율']= round(df_rate['율']*일괄적용율,4) #소수 4자리 반올림 
df_rate.loc[:,'회차']= [str(x).replace('회','') for x in df_rate['회차']]
df_rate=df_rate.astype({'회차':'int64'})
df_rate['회차T']= df_rate['회차'] 
df_rate['구분']=대상
df_rate['지점']=지점
df_rate=df_rate[['구분','회사','보종','율','회차','회차T','지점','회사율']]
df_rate.sort_values(by=['회사','보종','회차'],ascending=[False,True,True],inplace=True)

outFile=f'E:/temp/{지점}{보종} 지급율.xlsx' 
with pd.ExcelWriter(outFile) as writer:  
    df_rate.to_excel(writer, sheet_name='지급율',index=False)

In [5]:
## glad표준 엑셀 파일(아이리치)--임시용
지점='아이리치'
대상='사원'
일괄적용율=1.0
보종='손보'

path='E:/temp/'
in_file='book1.xlsx'
sheet='Sheet1'
c='회사 보종 환산율 1회 7회 8회 9회 10회 11회 12회 13회 14회 15회 지급계'
columns=c.split(' ')
df=pd.read_excel(path+in_file,sheet_name=sheet,header=0,names=columns)
df['회사'].fillna(method='ffill',inplace=True)
df.fillna(0,inplace=True )

### 보종명 변환

In [77]:
path='E:/temp/'
in_file='2018기준율-생보-변환전.xlsx'
sheet='Sheet1'

c='회사 보종 환산율 상품명 1회 1차년율 1차년회차 2차년율 2차년회차 3차년율 3차년회차 tot'
df=pd.read_excel(path+in_file,sheet_name=sheet)
df=df.fillna('')

In [78]:
b_list=['일반보장','일반종신','변액종신','연금','변액연금','저축','일반연금','변액적립','중저가',
        '보장','종신','보장2','보장3','보장4','경영인정기','적립','변액','양한방',
        '일반보장-경영','연금-7년','연금-5년','일반보장(경영인)','']
c_list=['보장성1','종신1','변액종신1','연금1','변액연금1','저축1','연금1','변액적립1','중저가1',
        '보장성1','종신1','보장성2','보장성3','보장성4','보장성1','변액적립1','변액연금1','양한방',
        '일반보장-경영','연금-7년','연금-5년','일반보장(경영인)','']
conv_dict=dict(zip(b_list,c_list))

In [79]:
lis=[5,15,25,35,45,55,65,75,85]
for i in lis:
#     print(i)
    df[i]=df[i].map(lambda x: conv_dict[x])

In [81]:
df.to_excel('e:/temp/2018기준율-생보-변환후.xlsx')